In [1]:
from pydantic import BaseModel
from typing import Optional
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS, DCTERMS
from pyfuseki import FusekiUpdate
from rdflib.namespace import XSD
from datetime import datetime
from src.schemas.settings import Settings
from pysolr import Solr
from src.function.catalog.markeCreateSparql import MakeCreateSparql
from src.schemas.catalog.instance import BfInstance
from src.schemas.catalog.work import BfWork, BfWorkEdit

In [2]:
settings = Settings()
solr = Solr(f'{settings.solr}/solr/catalog/', timeout=10)
fuseki = FusekiUpdate(settings.fuseki, 'bk') 


In [3]:
req = {
    "instanceOf": {
        "value": "https://bibliokeia.com/works/113"
    },
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/c",
            "label": "Editado"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        },
        "creationDate": "2024-04-10T16:19:54Z"
    },
    "resourceType": [
        {
            "value": "Instance",
            "label": "Instância"
        },
        {
            "value": "Print",
            "label": "Impresso"
        }
    ],
    "title": {
        "mainTitle": "O mestre de Petersburgo"
    },
    "physicalDetails": {
        "carrier": {
            "value": "http://id.loc.gov/vocabulary/carriers/nc",
            "label": "Volume"
        },
        "extent": "152 p.",
        "issuance": {
            "value": "http://id.loc.gov/vocabulary/issuance/mono",
            "label": "Unidade única"
        },
        "media": {
            "value": "http://id.loc.gov/vocabulary/mediaTypes/n",
            "label": "Não mediado"
        }
    },
    "provisionActivity": [
        {
            "place": "São Paulo",
            "agent": "Companhia das Letras",
            "date": "2019"
        }
    ],
    "image": {
        "cover": "https://m.media-amazon.com/images/I/51zh30TdqAL._SY445_SX342_.jpg"
    }
}

In [4]:
request = BfInstance(**req)
now = datetime.now()
request.adminMetadata.changeDate = now
request.adminMetadata.identifiedBy =  '58'  

In [5]:
from src.function.catalog.instance.graphInstance import MakeGraphInstance


def EditInstance(request):
    # Delete graph 
    instance_id = request.adminMetadata.identifiedBy
    uri = f"{settings.base_url}/instances/{instance_id}"
    print(uri)
    sparqlDropItem = f"DROP GRAPH <{uri}>"
    resDropWork = fuseki.run_sparql(sparqlDropItem)
    graph = MakeGraphInstance(request)
    sparql = MakeCreateSparql(graph, instance_id, "instances")
    response = fuseki.run_sparql(sparql) 

EditInstance(request)

https://bibliokeia.com/instances/58


In [20]:
request.responsibilityStatement

In [14]:
def DocInstance(request):

    work_id = f'work#{request.instanceOf.value.split("/")[-1]}'
    instance_id = f'instance#{request.adminMetadata.identifiedBy}'
    provisionActivity = request.provisionActivity[0]
    instanceOf = {
    'id': f'{instance_id}/instanceOf/{work_id}', 
    'uri': request.instanceOf.value, 
    'label': request.instanceOf.label
    } 
    print("DATE:", request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"))

    doc = {
        "id": instance_id,
        "creationDate": request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%SZ"), 
        # request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"),
        "type": [i.value for i in request.resourceType],
        "mainTitle": request.title.mainTitle,
        "subtitle": request.title.subtitle if request.title.subtitle != "" else None,
        "carrier": request.physicalDetails.carrier.label,
        "extent": request.physicalDetails.extent,
        "issuance": request.physicalDetails.issuance.label,
        "media": request.physicalDetails.media.label,
        "publicationAgent": provisionActivity.agent,
        "publicationDate": provisionActivity.date,
        "publicationPlace": provisionActivity.place,      
        # "serie": request.seriesStatement,
        "image": request.image.cover,
        "instanceOf": instanceOf
        }

    work = {
        "id": work_id,
        "hasInstance": {"add": doc }
    }
    # response = solr.add([work], commit=True)
    return work


work = DocInstance(request)
work

DATE: 2024-04-10T16:19:54


{'id': 'work#113',
 'hasInstance': {'add': {'id': 'instance#10',
   'creationDate': '2024-04-10T16:19:54Z',
   'type': ['Instance', 'Print'],
   'mainTitle': 'O mestre de Petersburgo',
   'subtitle': None,
   'carrier': 'Volume',
   'extent': '152 p.',
   'issuance': 'Unidade única',
   'media': 'Não mediado',
   'publicationAgent': 'Companhia das Letras',
   'publicationDate': 2019,
   'publicationPlace': 'São Paulo',
   'image': 'https://m.media-amazon.com/images/I/51zh30TdqAL._SY445_SX342_.jpg',
   'instanceOf': {'id': 'instance#10/instanceOf/work#113',
    'uri': 'https://bibliokeia.com/works/113',
    'label': None}}}}

In [15]:
settings = Settings()
solr = Solr(f'{settings.solr}/solr/catalog/', timeout=10)

In [16]:
work

{'id': 'work#113',
 'hasInstance': {'add': {'id': 'instance#10',
   'creationDate': '2024-04-10T16:19:54Z',
   'type': ['Instance', 'Print'],
   'mainTitle': 'O mestre de Petersburgo',
   'subtitle': None,
   'carrier': 'Volume',
   'extent': '152 p.',
   'issuance': 'Unidade única',
   'media': 'Não mediado',
   'publicationAgent': 'Companhia das Letras',
   'publicationDate': 2019,
   'publicationPlace': 'São Paulo',
   'image': 'https://m.media-amazon.com/images/I/51zh30TdqAL._SY445_SX342_.jpg',
   'instanceOf': {'id': 'instance#10/instanceOf/work#113',
    'uri': 'https://bibliokeia.com/works/113',
    'label': None}}}}

In [17]:
response = solr.add([work], commit=True)
response

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":207\n  }\n}'

In [3]:
request = BfWork(**req)
request

BfWork(resourceType=[BfElement(value='Work', metadata=None, label='Obra'), BfElement(value='Text', metadata=None, label='Texto')], adminMetadata=AdminMetadata(creationDate=None, changeDate=None, status=BfElement(value='http://id.loc.gov/vocabulary/mstatus/n', metadata=None, label='Novo'), descriptionConventions=BfElement(value='http://id.loc.gov/vocabulary/descriptionConventions/aacr', metadata=None, label='AACr'), identifiedBy=None), title=Title(mainTitle='Os que andam com os mortos', subtitle='Fábulas cruéis e outras estórias más'), variantTitle=None, language=[BfElement(value='por', metadata=None, label='Português')], genreForm=None, contribution=None, subject=None, classification=Classification(cdd='548.5', cutter='P552a'), note=None, summary=BfElement(value='Os que andam com os mortos, de Zemaria Pinto, é um desafio aos limites do conto enquanto gênero literário, na medida em que invade os territórios de outros géneros, estabelecendo uma conexão entre o contemporâneo e a tradição,

In [15]:
def DocWork(request):

    work_id = f'work#{request.adminMetadata.identifiedBy}'
    print("DATE:", request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"))

    doc = {
        "id": work_id,
        "creationDate": request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"),
        "changeDate": request.adminMetadata.changeDate.strftime("%Y-%m-%dT%H:%M:%S") if request.adminMetadata.changeDate else None,
        "type": [i.value for i in request.resourceType],
        # "content": request.content.label,
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification.cdd,
        "cutter": request.classification.cutter,
        "note": request.note,
        "summary": request.summary.value if request.summary else None,
        "tableOfContents": request.tableOfContents,
        # "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        # "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        # "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        # "geographicCoverage": [i.label for i in request.geographicCoverage] if request.geographicCoverage else None,
        "isPartOf": "Work"

    }
    # contribution
    if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = {"id": f"{work_id}/contribution/authority#{i.term.value.split('/')[-1]}",
                 "uri": i.term.value,
                 "label": i.term.label,
                 "role": i.role.value,
                 "roleLabel": i.role.label}
            contributions.append(c)
        doc['contribution'] = contributions

    # subject
    if request.subject:
        subjects = list()
        for i in request.subject:
            s = {"id": f"{work_id}/subject/authority#{i.term.value.split('/')[-1]}",
                 "type": i.type,
                 "uri": i.term.value,
                 "label": i.term.label,
                 "lang": i.lang
                 }
            subjects.append(s)
        doc['subject'] = subjects

    if request.genreForm:
        pass

    # responseSolr = solr.add([doc], commit=True)

    return doc


In [13]:
work = {
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resourceType": [
        {
            "value": "Work",
            "label": "Obra"
        },
        {
            "value": "Text",
            "label": "Texto"
        }
    ],
    "language": [
        {
            "value": "por",
            "label": "Português"
        }
    ],
    "classification": {
        "cdd": "548.5",
        "cutter": "O48"
    },
    "title": {
        "mainTitle": "Os que andam com os mortos"
    },
    "contribution": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/PersonalName/50",
                "label": "Coetzee, J. M."
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/aut",
                "label": "Autor"
            }
        }
    ],
    "subject": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/Topic/51",
                "label": "Literatura"
            },
            "lang": "por",
            "type": "Topic"
        }
    ]
}

request = BfWork(**work)
request

BfWork(resourceType=[BfElement(value='Work', metadata=None, label='Obra'), BfElement(value='Text', metadata=None, label='Texto')], adminMetadata=AdminMetadata(creationDate=None, changeDate=None, status=BfElement(value='http://id.loc.gov/vocabulary/mstatus/n', metadata=None, label='Novo'), descriptionConventions=BfElement(value='http://id.loc.gov/vocabulary/descriptionConventions/aacr', metadata=None, label='AACr'), identifiedBy=None), title=Title(mainTitle='Os que andam com os mortos', subtitle=None), variantTitle=None, language=[BfElement(value='por', metadata=None, label='Português')], genreForm=None, contribution=[Contribution(term=BfElement(value='https://bibliokeia.com/authority/PersonalName/50', metadata=None, label='Coetzee, J. M.'), role=BfElement(value='http://id.loc.gov/vocabulary/relators/aut', metadata=None, label='Autor'))], subject=[Subject(type='Topic', lang='por', term=BfElement(value='https://bibliokeia.com/authority/Topic/51', metadata=None, label='Literatura'))], cla

In [14]:
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = '13'

In [16]:
doc = DocWork(request)

DATE: 2024-04-08T16:30:50


In [18]:
response = solr.add([doc], commit=True)
response

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":90\n  }\n}'